In [1]:
#import packages
import pandas as pd
from sklearn import preprocessing 
import numpy

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import VarianceThreshold
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection

from sklearn import linear_model
from sklearn import ensemble
from sklearn import pipeline
from sklearn import metrics
from sklearn.feature_selection import f_regression

import seaborn as sns

In [2]:
#feature preprocessing and selection apply to all features. Therefore, the pipleine for that part is same for all outcomes
#different model strategies apply to different outcomes 

In [12]:
#read in data and drop unrelated variables 
df = pd.read_csv('C:\\Users\\azhang\\Documents\\county health rank 2018\\cleaned.csv')
df=df.drop(columns=['Unnamed: 0'])
df=df.drop(columns=['5-Digit FIPS Code'])
df=df.drop(columns=['state_county'])

In [13]:
df.percent_non_hispanic_black=df.percent_non_hispanic_black*df.percent_non_hispanic_white

df.percent_american_indian=df.percent_american_indian*df.percent_non_hispanic_white

df.percent_asian=df.percent_asian*df.percent_non_hispanic_white

df.percent_pacific_islander=df.percent_pacific_islander*df.percent_non_hispanic_white

df.percent_hispanic=df.percent_hispanic*df.percent_non_hispanic_white

df=df.drop(columns='percent_non_hispanic_white')

In [15]:
#test out feature selection by p value and correlations on whole data set

df.shape

(3141, 70)

In [16]:
#separate outcomes and feature
out = df.iloc[:,0]
#only focus on one outcome

#out.columns
feature=df.iloc[:,12:70]
#feature.columns

In [17]:
#build model on non missing Ys
notnans = pd.DataFrame(out.notnull()).all(axis=1)
Y_notnans = out[notnans]
X_notnans = feature[notnans]
#Y_notnans.describe()

# Split into 75% train and 25% test
X_train, X_val, Y_train, Y_val = train_test_split(X_notnans, Y_notnans,
                                                    train_size=0.75,
                                                    random_state=7)

C:\Users\azhang\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [18]:
#X_val.index

In [19]:
#scale y train
Y_train1=pd.DataFrame(Y_train)
yscale=preprocessing.StandardScaler()
Y_train2=yscale.fit_transform(Y_train1).ravel()

In [20]:
#before pipeline building, test different things out
#imputation
imputer=preprocessing.Imputer(missing_values=numpy.nan, strategy='mean', axis=0, verbose=0, copy=True)
X_train1=pd.DataFrame(imputer.fit_transform(X_train))
X_train1.columns=X_train.columns

#scaling using mean and standard deviation 
scaler = preprocessing.RobustScaler()
X_train2 = pd.DataFrame(scaler.fit_transform(X_train1))
X_train2.columns=X_train1.columns


In [21]:
#get f and p values for each feature
#from sklearn.feature_selection import f_regression
f_values, p_values = f_regression(X_train2, Y_train2)

p1=pd.concat([pd.DataFrame(X_train2.columns),pd.DataFrame(p_values.round(3))], axis=1)
p1.columns=['feature','p_val']

In [22]:
#get p values that are larger than 0.05 as potential candidates to exclude 
p1[p1['p_val']>0.05]

,feature,p_val
12,alcohol_impaired_driving_deaths,0.118
16,uninsured_children,0.198
18,other_primary_care_providers,0.074
32,nonwhite_white_residential_segregation,0.253
43,drinking_water_violations,0.490
49,percent_65_and_older,0.156


In [23]:
#get correlation between features 
#cov = numpy.cov(X_train2, rowvar=False)
#pd.DataFrame(cov)

corr=pd.DataFrame(X_train2.corr().abs().unstack().sort_values().drop_duplicates())

corr.reset_index(inplace=True) 
corr.columns=[ 'feature0','feature1','correlation']
corr[corr['correlation']>=0.7]

,feature0,feature1,correlation
1645,median_household_income,children_eligible_for_free_lunch,0.713382
1646,limited_access_to_healthy_foods,food_enviroment_index,0.737018
1647,children_in_poverty,food_insecurity,0.738777
1648,uninsured_children,uninsured,0.787408
1649,food_insecurity,food_enviroment_index,0.800587
1650,children_in_poverty,median_household_income,0.808936
1651,children_eligible_for_free_lunch,children_in_poverty,0.829947
1652,uninsured_adults,uninsured,0.989299
1653,food_insecurity,food_insecurity,1.000000


In [24]:
#because uninsured and children in poverty are highly associated with other variables from observation from train and test dataset, in validation, it will be dropped as well
#outcome in train/test: Y_train2
#feature in train/test: X_train_f
X_train_f=X_train.drop(columns=['uninsured'])
X_train_f=X_train_f.drop(columns=['children_in_poverty'])
X_train_f=X_train_f.drop(columns=['food_enviroment_index'])

#outcome in val: Y_val2
Y_val1=pd.DataFrame(Y_val)
yscale=preprocessing.StandardScaler()
Y_val2=yscale.fit_transform(Y_val1).ravel()

#feature in val: X_val_f
X_val_f=X_val.drop(columns=['uninsured'])
X_val_f=X_val_f.drop(columns=['children_in_poverty'])
X_val_f=X_val_f.drop(columns=['food_enviroment_index'])

In [25]:
#make pipeline with mean imputation and standard scaler, lassoCV feature selection, and linear regression as prediction model

#note:feature importance selection did not use recursive methods because it's not performing that well, and it takes too long

pipe_lr=pipeline.make_pipeline(preprocessing.Imputer(missing_values=numpy.nan, strategy='mean', axis=0),
              preprocessing.StandardScaler(), 
              #feature_selection.SelectFromModel(linear_model.LassoCV()),
              #feature_selection.RFECV(linear_model.LassoCV(), cv=10),
              linear_model.LinearRegression())
print(model_selection.cross_val_score(pipe_lr,  X_train_f, Y_train2, cv=10, scoring='r2').mean())
print(model_selection.cross_val_score(pipe_lr,  X_train_f, Y_train2, cv=10, scoring="neg_mean_squared_error").mean())

0.8314994406064435
-0.16767676040065216


In [26]:
#fit pipeline so that train set evaluation and coefficient extraction can be applied, and val set evaluation can also be applied 
pipe_lr.fit(X_train_f,Y_train2)

Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values=nan, strategy='mean', verbose=0)), ('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('linearregression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [28]:
coef=pd.DataFrame(pipe_lr.steps[2][1].coef_)
coef.columns=pd.DataFrame(out).columns
coef.index=X_train_f.columns
coef.to_csv('coef0C', sep='\t')

In [29]:
#evaluate on test r2
r2=metrics.r2_score(Y_val2, pipe_lr.predict(X_val_f))
#evaluate on test mean squared error
mse=metrics.mean_squared_error(Y_val2, pipe_lr.predict(X_val_f))

In [30]:
#sample size
Y_notnans.shape[0]

3071

In [31]:
evl=[r2,mse,Y_notnans.shape[0]]
evl1=pd.DataFrame(evl).T
evl1.columns=['r2','mse','sample_size']
evl1.index=pd.DataFrame(out).columns
evl1


,r2,mse,sample_size
premature_age_adjusted_mortality,0.81531,0.18469,3071.0


In [32]:

evl1.to_csv('evl0C', sep='\t')